### Esame di Internet of Things
# Un prototipo per un'applicazione web in Flask per la raccolta di dati
**Dario Gaudiuso**

# Introduzione
L'azienda <<Là di Cjastelan>> è un'azienda agricola a conduzione famigliare locata a San Osvaldo, Udine. L'azienda produce latte, derivati del latte, carni e insaccati e gastronomia. Per motivi fiscali, l'azienda ha bisogno di tener traccia di tutti i corrispettivi di cassa prodotti giornalmente. In altre parole, per ogni giorno lavorativo e per ciascuna delle tre casse dell'azienda, l'azienda ha bisogno di tracciare il totale ricavato dalla vendita di:
- prodotti con aliquota IVA 4%;
- prodotti con IVA 10% agricola;
- prodotti con IVA 10% commerciale;
- prodotti con IVA al 22%;
- totale complessivo della giornata.

Questi dati sono ricavati dai corrispettivi di cassa, cioè degli scontrini particolari che ogni cassa produce quando l'utente svolge l'operazione di chiusura a fine giornata lavorativa. Sommando opportunamente le voci dei corrispettivi, si ottengono le informazioni utili all'azienda.

Al momento l'azienda si serve di un Google Form per raccogliere i dati di un corrispettivo. Al momento della chiusura, l'operatore o l'operatrice di cassa compila manualmente i campi del Google Form per trascrivere le voci del corrispettivo. A fine trimestre, una persona scarica tutte le risposte del form e aggrega manualmente i dati su un foglio elettronico Excel. Come si può immaginare, questo modo di svolgere l'operazione è prono a errori, sia al momento della compilazione del Google Form che, poi, al momento dell'aggregazione.

L'utilizzo di un'applicazione web rappresenta una comoda soluzione per il problema dell'azienda, perché permette di mettere in atto diverse strategie per il riconoscimento degli errori di inserimento già nel tentativo di aggiunta di un nuovo record, garantendo che nel dataset non arrivino dati palesemente anomali. Inoltre, usando metodi di acquisizione automatica, come per esempio l'estrazione delle voci dalla fotografia del corrispettivo tramite motore di riconoscimento ottico dei caratteri, possiamo anche alleggerire il carico cognitivo dell'utente e velocizzare l'inserimento.

Scopo del progetto è progettare e sviluppare un prototipo per un'applicazione web con paradigma REST che consenta a un utente autenticato le operazioni di inserimento, modifica e visualizzazione di dati riguardanti corrispettivi e mercati. La parte di inserimento deve permettere all'utente di scattare una foto al corrispettivo e l'applicazione dovrà compilare automaticamente i campi del form con le informazioni lette dalla fotografia. L'applicazione sarà sviluppata usando Flask, un micro-framework per applicazioni web in Python.

## Funzionalità

Dividiamo il dominio applicativo in due macro-aree: le funzionalità legate ai corrispettivi e le funzionalità legate ai mercati. 

L'applicazione deve permettere a un'utente autenticato la registrazione, all'interno di un database, dei corrispettivi di cassa giornalieri tramite compilazione di un web form. Se l'utente sta usando un dispositivo mobile (smartphone), l'applicazione deve permettere all'utente di fotografare lo scontrino per compilare automaticamente il form con le voci estratte dalla foto. All'utente autenticato con privilegi di admin è permesso anche visualizzare tutti i corrispettivi presenti nel database e la modifica dei record.

L'applicazione deve garantire il più possibile che i dati inseriti dagli utenti siano corretti.

La gestione dei mercati è delegata ai soli utenti autenticati con privilegi di admin; questi possono inserire nuovi mercati e modificare quelli esistenti.

## Errori e strategie di prevenzione

Durante lo studio del dominio, nell'analizzare i dati precedentemente raccolti dall'azienda (dal 2018 a oggi) e le modalità di raccoglimento degli stessi, sono emersi numerosi errori di inserimento: circa il 10% dei record nel dataset erano sbagliati. Dovendo minimizzare la possibilità di inserimento di dati errati, occorre sviluppare delle strategie verticali nell'architettura del progetto che garantiscano il più possibile la correttezza dei dati inseriti dall'utente, lavorando sia a livello utente, mediante la progettazione di interfacce utente efficaci, sia a livello applicativo, mediante lo sviluppo di controlli sui dati prima dell'inserimento nel database, che a livello di base di dati, con vincoli intrarelazionali e interrelazionali tra i dati. 

Analizzando gli errori presenti nei dati raccolti finora, emerge che gli errori più comuni sono errori di copiatura in cui una cifra viene letta al posto di una simile, per esempio si inserisce <<1>> invece di <<7>> e viceversa. Questo tipo di errore può essere parzialmente evitato grazie all'acquisizione automatica delle voci tramite OCR; si dice parzialmente perché, a posteriori, si è visto che nel nostro dominio anche l'OCR compie spesso lo stesso tipo di errore, scambiando le cifre <<1>> e <<7>> e <<8>> e <<0>>. Non potendo prevenrire questo errore, bisogna allora concentrarsi nell'individuarlo quando accade per permettere all'utente di correggerlo immediatamente. La strategia per lìindividuazione dell'errore di copiatura consiste nello sfruttare la ridondanza dei dati: i corrispettivi presentano, oltre al subtotale di ogni reparto, il totale giornaliero incassato, cioè la somma di tutti i subtotali. Invece di calcolare questo totale complessivo, lo raccogliamo insieme alle altre voci e, prima di autorizzare l'inserimento, verifichiamo che il totale inserito coincida con il totale calcolato sommando tutti i subtotali. Qualora i due valori non coincidano, vuol dire che, da qualche parte nei subtotali o nel totale, è stato fatto un errore di copiatura. Questo controllo avviene a livello di applicazione.

Un altro errore comune è l'inserimento di scontrini duplicati. Alcune volte lo stesso scontrino viene inserito più di una volta, altre volte vengono inseriti gli stessi totali di reparto ma su date diverse o mercati diversi; questo caso potrebbe certamente essere legittimo, perché è possibile che due mercati diversi abbiano esattamente gli stessi subtotali per ogni reparto, ma molto più spesso si tratta del tentativo di correzione di un errore di inserimento precedente: il Google Form, infatti, non permette di modificare i record già inseriti, quindi l'utente tende a inserire un nuovo record quando si accorgem in ritardo, di aver sbagliato qualcosa. Possiamo agire su questo errore a livello di applicazione, chiedendo conferma all'utente prima di effettuare l'inserimento se l'applicazione si accorge che nella base di dati esiste già un record dove tutti i valori dei reparti coincidono con quelli dell'inserimento che si vuole effettuare. L'inserimento non può essere impedito a livello di database perché, seppur etremamente improbabile, non è detto che non possano esserci due record diversi, entrambi corretti, con gli stessi valori sui reparti.

Un altro problema è dato dalle date e dai numeri scritti in modo sbagliato e incoerente, per esempio date inserite come <<1/1/0024>>, oppure  totali scritti come <<35, 01>> (con uno spazio dopo la virgola) o anche <<7,345>>, con tre cifre decimali. Questo problema si previene facilmente, introducendo una base di dati che impone un tipaggio forte dei dati. Lato utente, progettiamo l'interfaccia in modo che suggerisca all'utente il formato dei dati, per esempio usando un selettore a calendario per le date, e che imponga il corretto formato di ogni dato inserito dall'utente. Lato applicazione, bisogna ovviamente effettuare tutti i controlli e le conversioni sui dati ricevuti dall'utente.

Le date pongono problemi più seri quando sono incoerenti col mercato a cui si riferiscono. Per esempio, succede spesso di confondere la numerazione dei mesi e inserire "1/10/24" per un corrispettivo di settembre. In questo modo, slitta il giorno della settimana e ci si trova, per esempio, con un record che fa riferimento a un mercato che, regolarmente, si svolge di martedì e che, invece, stando alla data inserita, si è svolto di sabato. Questo potrebbe anche non essere un errore, nel caso eccezionale in cui un mercato sia stato spostato per un'unica data. Capita anche di trovare scontrini con date del futuro, per esempio col 2026 come anno. Il vincolo per cui uno scontrino non possa fare riferimento a una data nel futuro può essere imposto a livello di database e controllato ancor prima a livello di applicazione, mentre il controllo della coerenza tra mercato e giorno della settimana deve essere effettuato a livello applicativo e gestito a seconda dei casi.

L'ultimo errore riguarda le casse: si trovano giorni in cui la stessa cassa è stata usata su due mercati diversi oppure mercati in cui sono state usate due casse contemporaneamente. Per le modalità con cui lavora l'azienda, questa situazione è impossibile ed è indice di un errore di inserimento o sul mercato o sulla cassa. Nel Google Form, infatti, quando si chiede all'utente di inserire il mercato cui fa rimento il corrispettivo, i mercati sono accoppiati nella stessa voce col giorno di svolgimento: le voci tra cui l'utente deve scegliere sono del tipo "Centro - Lunedì", "Centro - Martedì", "Passons - Martedì", ecc. Accade, quindi, di confondere i mercati che si svolgono lo stesso giorno, per esempio "Centro - Sabato" viene inserito al posto di "Cormor - Sabato" perché l'attenzione dell'utente è catturata maggiormente dal giorno della settimana. Questa situazione può essere evitata a livello di database, imponendo come vincolo di chiave primaria la data e il mercato e, a livello di interfaccia utente, separando l'informazione del mercato dall'informazione del giorno della settimana in cui si è svolto.

# Architettura dell'applicazione

## Database e ORM
Come DBMS abbiamo scelto di utilizzare SQLite perché è il più leggero e semplice da configurare e mette a disposizione tutte le funzionalità che servono per database piccoli e con pochi accessi: l'applicazione, infatti, sarà usata al più da quattro persone contemporaneamente e si prevede l'inserimento di circa due corrispettivi al giorno; si tratta di un carico di lavoro veramente irrisorio, perfetto per sfruttare tutta la semplicità che SQLite offre senza inciampare nelle sue limitazioni. Per interfacciare l'applciazione Flask a SQLite, utilizziamo l'ORM SQLAlchemy. Fortunatamente, Flask è dotato della libreria `Flask-SQLAlchemy` che semplifica di molto la configurazione.

L'inizializzazione del database è effettuata nel file `database.py`, dove si trovano anche i modelli per creare le tabelle e, contestualmente, le classi che mappano a esse. La classe `Base`, che eredita da `DeclarativeBase`, è un artificio di SQLAlchemy che serve per ottenere l'istanza della classe `MetaData`, la quale contiene tutte le infromazioni per il mapping tra tabelle SQL e rappresentazione a oggetti. Ereditando da `Base`, i modelli popolano automaticamente l'istanza di `Metadata`.

L'oggetto `db` creato dalla libreria `Flask-SQLAlchemy` espone due interfacce: `db.Model`, per creare i modelli, e `db.session` per instaurare la connessione al database. `db.session` è già configurato dalla libreria come `scoped_session`, che è la versione thread-safe dell'oggetto `session` di SQLAlchemy: in altre parole, se una stessa richiesta deve fare più accessi al database, non viene creata una nuova connessione per ogni accesso ma si riutilizza la prima connessione instaurata dalla richiesta.

Dopo aver creato le tabelle, la funzione `init_db()` attiva il vincolo di chiave esterna, che in SQLite è disabilitato di default per questioni di retrocompatibilità, e popola UTENTI e MERCATI con alcuni record di prova necessari per far funzionare l'applicazione.

### Struttura del database

Usiamo tre tabelle:
- Utenti: per gestire le persone che possono usare l'applicazione e i relativi privilegi
- Corrispettivi: dove verranno conservati i dati inseriti
- Mercati: contiene informazioni su tutti i mercati svolti dell'azienda

La struttura delle tabelle è la seguente, dove una sottolineatura indica la chiave primaria e il corsivo indica una chiave esterna:

UTENTI(<u>username</u>, password, is_admin)  
CORRISPETTIVI(<u>data, *mercato*</u>, timestamp, *inserito_da*, cassa, *giorno_mercato*, quantità1, reparto1, quantità2, reparto2, quantità3, reparto3, quantità4, reparto4, quantità5, reparto5)  
MERCATI(<u>nome, giorno</u>, is_evento, is_attuale)

CORRISPETTIVI(timestamp) è il timestamp del momento in cui viene registrato il corrispettivo. Registriamo questa informazione perché, in caso di errori, è comodo sapere in che momento è stato effettuato l'inserimento (possiamo infatti capire se si tratta di un inserimento in blocco di più scontrini o quanto tempo è passato dal momento in cui si è svolto il mercato al momento in cui viene inserito il relativo corrispettivo). Il valore non è inserito dall'utente, ma compilato automaticamente dal web server.
MERCATI(giorno) è il giorno della settimana in cui si svolge il mercato registrato; MERCATI(is_attuale), invece, indica se il mercato si svolge nel presente o se è stato dismesso. Dovendo conservare lo storico di tutti i corrispettivi, non possiamo semplicemente eliminare un mercato che non si svolge più, ma dobbiamo in qualche maniera conservarlo all'interno della base di dati.

### Vincoli

Tutti gli attributi hanno il vincolo NOT NULL tranne MERCATI(is_attuale).

Nella relazione CORRISPETTIVI, *inserito_da* è chiave esterna riferita a UTENTI(username) e (*mercato*, *giorno_mercato*) è chiave esterna riferita a MERCATI.

Vincoli intrarelazionali su CORRISPETTIVI:
 1. *data* non può essere una data successiva a quella indicata da *timestamp*
 2. I *reparti* devono essere decimali a due cifre non negativi
 3. Le _quantità_ devono essere interi non negativi

Vincoli intrarelazionali su MERCATO:

 4. Se *is_evento* è TRUE, allora *is_attuale* deve essere NULL; altrimenti *is_attuale* non può essere NULL
 4. Se il mercato non è attuale (_is_attuale_ è FALSE), il nome deve terminare con la stringa `_old`, es. <\<Centro_old>>

I vincoli 1., 3. e 4. sono stati implementati a livello di DDL. Il vincolo sul numero di cifre decimali di CORRISPETTIVI(reparto1, ..., reparto5) non può essere gestito a basso livello giacché SQLite non possiede il tipo DECIMAL. Anche l'aggiunta automatica della stringa "_old" al nome di un mercato non più attuale non può essere gestita a livello di DBMS per limitazioni di SQLite sui trigger: gli inserimenti e le modifiche dei mercati non più attuali dovranno quindi essere opportunamente gestiti a livello di applicazione.

## Applicazione Flask

### Inizializzazione
Il file `__init__.py` funge da entry point per l'applicazione. Al suo interno, usando una _factory function_, si crea e restituisce l'istanza dell'applicazione, iniziallizzando le necessarie variabili ambientali. L'opzione `instance_relative_config=True` serve a dire che le variabili ambientali devono essere impostate leggendo un file che si trova nella cartella `instance`, all'esterno del package dell'applicazione. Questo serve a isolare i segreti e il database dal resto dell'applicazione.

### Blueprint
Modularizziamo l'applicazione creando, per ogni funzionalità dell'applicazione, un blueprint che raggruppa tutte le rotte necessarie per ottenere quella funzionalità. In totale l'applicazione si serve di quattro blueprint. Tutte le rotte che assolvono funzionalità per l'autenticazione degli utenti sono raggruppate nel blueprint `auth`, le rotte che permettono l'inserimento dei corrispettivi sono nel blueprint `corr`, tutta la logica dell'OCR è nel blueprint `ocr` e, infine, le rotte per la gestione dei mercati sono nel blueprint `mercati`.

### Autenticazione
L'autenticazione è gestita in maniera veramente basilare, verificando se la password inserita dall'utente fa match con la password registrata nel database sotto lo stesso username. Per implementare le funzionalità di autenticazione, usiamo due oggetti speciali di Flask, `g` e `session`. Il primo è un namespace globale all'interno del contesto della richiesta, cioè permette di condividere informazioni tra diverse funzioni all'interno del contesto della medesima richiesta. Il secondo, invece, permette di persistere informazioni tra più richieste effettuate dallo stesso client. Usiamo `session` per ricordare che l'utente è autenticato.

L'oggetto `g` invece entra in gioco per implementare la limitazione all'accesso alle funzionalità dell'app ai soli utenti autenticati. Definiamo il decoratore `login_required()` da applicare alle rotte delle quali vogliamo limitare l'accesso. Esso controlla se l'utente è attivo nella sessione, quindi carica le sue informazioni in `g`, mettendole a disposizione di tutte le funzioni che verranno invocate dalla vista (la funzione che viene invocata quando si raggiunge una rotta). Se l'utente non è autenticato (cioè non è presente nella sessione), si effettua un redirect sulla rotta del login. Le sezioni dedicate solo all'admin sono invece protette dal decoratore `admin_required`, che funziona in modo del tutto simile.

Se l'utente non ha i privilegi di admin, dopo un'autenticazione terminata con successo, viene dirottato direttamente sulla pagina per l'inserimento dei corrispettivi. Se invece l'utente è un admin, viene dirottato in una pagina in cui può scegliere quale operazione effettuare.

### Inserimento dei corrispettivi
Per aumentare l'usabilità, l'ordine dei giorni della settimana viene modificato dinamicamente con la funzione JavaScript `sort_giorni()`. Questa reagisce alla compilazione del campo *Mercato* del form e porta in cima alla lista delle opzioni disponibili per il campo *giorno* i giorni in cui si svolge il mercato selezionato. Per far comunicare (condividere le variabili) il server con il modulo JavaScript che lavora lato client, dichiariamo le variabili da condividere all'interno di un tag `<script>` nel corpo del template e usiamo il filtro `|tojson` messo a disposizione da Jinja.

La funzionalità di inserimento effettua sui corrispettivi tutti i controlli di cui si è discusso in precedenza, impedendo l'inserimento in caso di errori palesi e dando un warning quando si stanno inserendo dati che potrebbero essere sbagliati, ma non lo sono necessariamente.

A livello di codice, alcune idiosincrasie di SQLAlchemy sono state segnalate con commenti nelle viste. La funzione `validate_input()` si assicura che i dati che arrivino dal client siano corretti dal punto di vista del DDL; essa crea, quindi, l'istanza della classe dichiarativa SQLAlchemy `Corrispettivi` da ritornare alla vista principale, la quale, in assenza di errori, ne tenta l'inserimento nella base di dati, dirottando in una pagina di conferma nel caso di successo o visualizzando un errore altrimenti.

### OCR

Le funzionalità legate all'OCR sono ancillari alle funzionalità legate all'inserimento dei corrispettivi. Solo nel caso in cui l'utente stia usando l'applicazione tramite smartphone, permettiamo l'acquisizione di una fotografia dello scontrino per l'estrazione automatica delle sue voci. La fotografia viene ridimensionata, inviata al server, preprocessata e letta dal motore EasyOCR. Il risultato della lettura viene poi strutturato dal server e restituito al client. La procedura è gestita lato client tramite AJAX effettuata con l'API `fetch()`; la logica è nel modulo `corr.js`. Usando l'AJAX, permettiamo al server di modificare dinamicamente le informazioni presenti sulla pagina (il valore dei campi del form) senza ricaricare l'intera pagina, il che avverrebbe anche al prezzo di resettare i campi del form che l'OCR non ha letto ma che l'utente ha compilato. Per tutti i dettagli sul funzionamento dell'OCR, rimando al notebook `relazione_iot.ipynb`.

### Inserimento dei mercati
Per rendere migliore l'esperienza utente, facciamo in modo che tutte le operazioni di visualizzazione, creazione e modifica dei mercati siano possibili all'interno di una sola pagina, cioè ci serviamo di una sola interfaccia utente per effettuare tutte le operazioni sui dati.

Il modo più intuitivo per visualizzare tutti i mercati è quello di utilizzare una tabella, ma in HTML il tag `<form>` non può essere uno dei discendenti del tag `<table>`. Utilizziamo, quindi, una funzionalità introdotta da HTML5 che permette di dichiarare un form vuoto esterno alla tabella, dandogli un id, e usare nella tabella elementi `<input>` non inseriti in nessun form, ma collegati al form esterno precedentemente dichiarato tramite l'attributo `form=<id del form>`.

L'implementazione dell'operazione di inserimento non ha nulla di particolare, a parte gli accorgimenti per implementare a livello di applicazione i vincoli intrarelazionali che, come si è discusso prima, non possono essere implementati a livello di database. Per visualizzare e nascondere il form di inserimento nell'interfaccia ci serviamo dell'oggetto `session` e di una rotta specifica che imposta nella sessione le variabili necessarie.

L'operazione di modifica si appoggia all'operazione di inserimento per la logica e la validazione dell'input e sfrutta anch'essa l'oggetto `session` sia per adeguare l'interfaccia, sia per recuperare la giusta istanza di `Mercati` dall'ORM. Per identificare quale riga della tabella si vuole modificare, usiamo la funzionalità di Flask che ci permette di passare argomenti nell'endpoint.

Il controllo dell'oggetto condiviso `session` ci permette anche di rendere mutualmente esclusive le operazioni di inserimento e modifica. Sfruttando la stessa interfaccia è, infatti, teoricamente possibile aprire sia il form di inserimento che quello di modifica; questo, però, potrebbe portare a risultati inconsistenti, quindi lo proibiamo controllando lo stato della sessione quando si vuole cominciare una delle due operazioni.